# Implementing an untyped Lambda Calculus


To introduce `metadsl` we will show how to create a working version of the untyped lambda calculus.

This will let us create the some basic combinators, verify that they before as expected under equiavelency.

Then we can also create an implementation of arithmetic on natural numbers, using church encoding and show how to convert
between these and Python's integers.


## Terms
The lambda calculus might sound abstract and odd, but if you know how to write functions in Python you already have mastered it!
The idea is a very simple language that only has functions that take in one argument.
Nothing, else, which seems a bit silly maybe, and it is not very useful by itself, but when it is integrated into a system with other data
types, it becomes a useful so that we can represent functions.


First, let's create a concept of a "variable" that can optionally have a name. It should only be equal to itself, so that we don't have to worry about shadowing variable names:

In [1]:
import dataclasses
import typing


@dataclasses.dataclass(eq=False)
class Variable:
    name: typing.Optional[str] = None
    
    
    def __str__(self):
        return self.name or str(id(self))
        
    def __repr__(self):
        return f"Variable({str(self)})"
        
x1 = Variable("x")
x2 = Variable("x")

assert x1 == x1
assert x1 != x2
assert hash(x1) != hash(x2)

Now, let's build our one and only key abstraction, a lambda term.
You should be able to call it and also create it by having a body and a variable. 

In [2]:
import metadsl


class LambdaTerm(metadsl.Instance):
    @metadsl.call(lambda self, arg: LambdaTerm)
    def __call__(self, arg: "LambdaTerm") -> "LambdaTerm":
        ...

@metadsl.call(lambda variable, body: LambdaTerm)
def abstraction(variable: Variable, body: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda variable: LambdaTerm)
def from_variable(variable: Variable) -> LambdaTerm:
    ...

def create_abstraction(fn: typing.Callable[[LambdaTerm], LambdaTerm]) -> LambdaTerm:
    arg = Variable()
    return abstraction(arg, fn(from_variable(arg)))

We can construct the identity function

In [3]:
@create_abstraction
def identity(x):
    return x

In [4]:
identity

LambdaTerm(__call__=Call(abstraction, (Variable(4515780088), LambdaTerm(__call__=Call(from_variable, (Variable(4515780088),))))))

and then call it on a term:

In [5]:
y_term = from_variable(Variable("y"))
called_identity = identity(y_term)
called_identity

LambdaTerm(__call__=Call(__call__, (LambdaTerm(__call__=Call(abstraction, (Variable(4515780088), LambdaTerm(__call__=Call(from_variable, (Variable(4515780088),)))))), LambdaTerm(__call__=Call(from_variable, (Variable(y),))))))

This is a bit of a verbose way to print these.

Let's create a custom function that walks the tree and converts them to a string:

In [6]:
pretty_print = metadsl.Replacements()


@metadsl.call(lambda s: LambdaTerm)
def string_term(s: str) -> LambdaTerm:
    ...


@pretty_print.register(None)
def pp_variable(variable: Variable):
    return (
        from_variable(variable),
        lambda: string_term(str(variable))
    )

@pretty_print.register(None, None)
def pp_abstraction(variable: Variable, body: str):
    return (
        abstraction(variable, string_term(body)),
        lambda: string_term(f"(λ{variable}.{body})")
    )

@pretty_print.register(None, None)
def pp_application(fn: str, arg: str):
    return (
        string_term(fn)(string_term(arg)),
        lambda: string_term(f"({fn} {arg})"),
    )

In [7]:
pretty_print(called_identity)

LambdaTerm(__call__=Call(string_term, ('((λ4515780088.4515780088) y)',)))

To make this useful, we need to define some semantics on these.

What's a useful thing to do with the lambda calculus?

Well it would be nice to actually be able to "compute" these values by replacing all instances of a variable with it's argument.

So let's replace all instances of calling a variable by replacing the arg inside of it:

In [16]:
replace_beta_reduce = metadsl.Replacements()


@replace_beta_reduce.register(None, LambdaTerm, LambdaTerm)
def beta_reduce(var: Variable, body: LambdaTerm, arg: LambdaTerm):
    # replaces all instances of var with arg 
    replacer = metadsl.expression_replacer(((from_variable(var), arg),))
    
    return (
        abstraction(var, body)(arg),
        lambda: replacer(body)
    )

Now let's combine these two, to execute them in sequence. First we want to beta reduce, until there is no more to beta reduce, then
we wanna turn that into a pretty print representation:

In [19]:
def execute(expr: LambdaTerm) -> str:
    return pretty_print(replace_beta_reduce(expr))

In [20]:
print(execute(called_identity))

LambdaTerm(__call__=Call(string_term, ('y',)))


This looks allright, but before we do anything serious, let's deal with the annoying issue of these variable names. They aren't much fun to read! Let's fix this by implementing [α-conversion](https://en.wikipedia.org/wiki/Lambda_calculus#%CE%B1-conversion). The basic idea is that given some lambda expression, we rename all the variables to make them standard. Let's create a bunch of global variables, and then create a function that will transform an expression into a normal form using them:

In [23]:
GLOBAL_VARS = [Variable(chr(i)) for i in range(ord('a'), ord('a') + 100)]
CURRENT_GLOBAL_VAR = 0

In [24]:
GLOBAL_VARS[0]

LambdaTerm(__call__=Call(from_variable, (Variable(a),)))

The idea is that we will map each variable we find in the expression to one of the global variables:

In [ ]:
alpha_convert_replacement = metadsl.Replacements()

@alpha_convert_replacement.register(None, LambdaTerm)
def alpha_convert(var: Variable, body: LambdaTerm):
    def replace(var=var, body=body):
        # we have already replaced this one
        if var in GLOBAL_VARS:
            return abstraction(var, body)

        global CURRENT_GLOBAL_VAR

        new_var = GLOBAL_VARS[CURRENT_GLOBAL_VAR]
        CURRENT_GLOBAL_VAR += 1

        metadsl.expression_replacer((
            (from_variable(var), from_variable() ),
        ))
        replace_var = metadsl.replace_all(lambda: (LambdaTerm(var), (lambda: LambdaTerm(new_var))))
        
        return abstraction(new_var, replace_var(body)) 
    
    return (
        abstraction(var, body),
        replace
    )

In [ ]:
identity

In [ ]:
alpha_convert(identity)

Now let's add this to our execution step:

In [ ]:
def execute(expr: LambdaTerm) -> str:
    global CURRENT_GLOBAL_VAR
    CURRENT_GLOBAL_VAR = 0
    return pretty_print(alpha_convert(beta_reduce(expr))).__value__

In [ ]:
print(execute(identity(identity)))